# Mining - Drilling data pipeline solution - Sukari

This **solution** will 
1. Read Mining and Drilling excel files that we currently input them manually daily.
2. Do some data cleaning. 
3. Store cleaned data in a proper way in a database.

In [65]:
# import libraries

import glob
import numpy as np
import pandas as pd
from datetime import time, date
import sqlalchemy as sa
import warnings

# ignore some warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

### 1 - Read Excel Files

#### 1 - A - Mining Data
There are 3 excel sheets for Mining:
1. Daily Dispatch Master.
    - This is our main excel sheet, it has data for **Loads**, **Hours** and **SMU**.
    - Loads and Hours are stored as aggregates for every one hour interval, SMU is recorded at shift start and end.
2. Daily Production Report.
    - Will extract the monthly truck factor from it.
3. Daily Production Performance
    - Has distances for excavators recorded as aggregates per shift.

In [66]:
# Read dispatch master
def read_dispatch():
    
    dispatch_path = 'dispatch/*.xlsx'
    dispatch_files = glob.glob(dispatch_path)
    dispatch_df = pd.DataFrame()
    print('Reading Dispatch Files...\n')
    if len(dispatch_files) < 1 :
        raise ValueError('No files to read!')
    else : 
        print('dispatch files number ={}\n'.format(len(dispatch_files)))
        for file in dispatch_files:
            print("Reading {}".format(file))
            df = pd.read_excel(io=file, sheet_name="Fix Database",  usecols="B:W")
            print("Columns = {}".format(len(df.columns)))
            dispatch_df = pd.concat([dispatch_df, df], ignore_index=True, axis=0)
            print("Master df Columns = {}".format(len(dispatch_df.columns)))
            print("finished {}\n".format(file))
            if len(dispatch_df.columns) > 22:
                 raise ValueError('Cols are more than 22, check excel sheet format')
    print('last dispatch date =', dispatch_df['DATE'].tail(1))
    print('\nDispatch Files Reading Ended Successfully...')
    
    return dispatch_df

In [67]:
# Read performance master
def read_performance():
    
    performance_path = 'performance/*.xlsx'
    performance_files = glob.glob(performance_path)
    performance_df = pd.DataFrame()
    print('\nReading Performance Files...\n')
    if len(performance_files) < 1 :
        raise ValueError('No files to read!')
    else : 
        print('performance files number =', len(performance_files))
        for file in performance_files:
            df_ds = pd.read_excel(io=file, sheet_name="Distance (dsns)", skiprows=2, usecols="B,C,D", header=None, names=['DATE','EXC','distance'])
            df_ds['shift'] = 'day'
            df_ns = pd.read_excel(io=file, sheet_name="Distance (dsns)", skiprows=2, usecols="O,P,Q", header=None, names=['DATE','EXC','distance'])
            df_ns['shift'] = 'night'
            df = pd.concat([df_ds, df_ns], ignore_index=True, axis=0)
            performance_df = pd.concat([performance_df, df], ignore_index=True, axis=0)
            print("finished {}".format(file))
            print("Columns = {}".format(len(performance_df.columns)))
            if len(performance_df.columns) != 4:
                 raise ValueError('Cols are not equal to 4, check excel sheet format')
    print('last performance date =', performance_df['DATE'].tail(1))
    print('\nPerformance Files Reading Ended Successfully...')
    
    return performance_df

In [68]:
# Read dpr
def read_dpr():
    
    dpr_path = 'dpr/*.xlsx'
    dpr_files = glob.glob(dpr_path)
    dpr_df = pd.DataFrame()
    print('\nReading dpr Files...\n')
    if len(dpr_files) < 1 :
        raise ValueError('No files to read!')
    else : 
        print('dpr files number =', len(dpr_files))
        for file in dpr_files:
            df = pd.read_excel(file, sheet_name='Data Sheet', usecols="A,H,L,M,N")
            dpr_df = pd.concat([dpr_df, df], ignore_index=True, axis=0)
            print("finished {}".format(file))
            print("Columns = {}".format(len(dpr_df.columns)))
            if len(dpr_df.columns) != 5:
                 raise ValueError('Cols are not equal to 4, check excel sheet format')
    print('last performance date =', dpr_df['Date'].tail(1))
    print('\nPerformance Files Reading Ended Successfully...')
    
    return dpr_df

In [69]:
# Read reason codes
def read_reason_codes():
    
    reason_codes_path = 'reason_codes/reason_codes.csv'
    
    print('\nReading Reason codes...\n')
       
    reason_codes = pd.read_csv(reason_codes_path)

    print('\nReason codes Reading Ended Successfully...')
    
    return reason_codes

#### 1 - B - Drilling Data
There are 2 excel sheets for Drilling:
1. Daily Drilling Data.
    - This is our main excel sheet, it has meters drilled aggregated for each hole.
2. Rigs Data.
    - Has the rigs hours aggregated per shift

In [70]:
# Read daily drilling data
def read_drilling():
    
    daily_data_path = 'drilling/*.xlsx'
    daily_data_files = glob.glob(daily_data_path)
    meters_df = pd.DataFrame()
    rigs_hours_df = pd.DataFrame()
    print('\nReading drilling Files...\n')
    if len(daily_data_files) < 1 :
        raise ValueError('No files to read!')
    else :
        print('daily data files number =', len(daily_data_files))
        for file in daily_data_files :         
            meters = pd.read_excel(io=file, sheet_name = 'DailyDrillingData', usecols= "E:V")
            meters_df = pd.concat([meters_df, meters], ignore_index =True, axis=0)
            rigs_hours = pd.read_excel(io=file, sheet_name = 'Rigs Data', usecols="C:BQ")
            rigs_hours_df = pd.concat([rigs_hours_df, rigs_hours], ignore_index =True, axis=0)
    print('last meters date =', meters_df['Date'].tail(1))
    print('last rigs_hours date =', rigs_hours_df['Date'].tail(1))
    print('\nDrilling Files Reading Ended Successfully...')
    
    return meters_df, rigs_hours_df

In [71]:
# Extraction Process
def extract_data():
    print('*** START DATA EXTRACTION ***\n')
    
    dispatch_df = read_dispatch()
    performance_df = read_performance()
    dpr_df = read_dpr()
    meters_df, rigs_hours_df = read_drilling()
    reason_codes_df = read_reason_codes()
    
    print('\n***DATA EXTRACTION ENDED ***\n')
    
    return dispatch_df, performance_df, dpr_df, meters_df, rigs_hours_df, reason_codes_df

### 2 - Data Cleaning

#### 2 - A - General data cleaning

#### 2 - A - Dispatch master cleaning

In [72]:
def general_clean(dfs):
    
    new_dfs = [df.copy() for df in dfs[:len(dfs)-1]]
    # clean column names
    for df in new_dfs:
        df.columns = df.columns.str.lower().str.replace(' ','_')

    # create a new list of dfs
    
    new_dfs = [df[df['date'].isnull() == False] for df in new_dfs]

    return new_dfs

In [73]:
# Transform dispatch
def transform_dispatch(df,x):
    
    print("/// dispatch transformation started ///")

    dispatch_df = df[0].copy()

    dispatch_df.dropna(how='all', inplace=True)

    print('Converting production values to numeric...')
    dispatch_df['production_value'] = pd.to_numeric(dispatch_df['production_value'], errors='coerce')
    errors = dispatch_df[dispatch_df['production_value'].isna()]
    errors_index = errors.index.tolist()
    n_errors = len(errors_index)
    dispatch_df.drop(index=errors_index, inplace=True)
    dispatch_df = dispatch_df[dispatch_df['production_value'] > 0]
    print('Found and dropped {} errors'.format(n_errors))

    # convert codes to integers
    dispatch_df['reason_code'] = dispatch_df['reason_code'].astype('Int64')

    # adjust time col
    dispatch_df['time'] = dispatch_df['time'].str.split('-').str[0]
    dispatch_df['time'] = pd.to_numeric(dispatch_df['time'], errors='coerce')
    dispatch_df['time'] = dispatch_df['time'].replace(np.nan, 0).round()

    # adjust date col
    dispatch_df['date'] = pd.to_datetime(dispatch_df['date'])

    # assign dtypes
    dispatch_df = dispatch_df.astype({'shift' : 'string', 
                                'operator_name':'string',
                                'crew':'string',
                                'reason_activity':'string',
                                'engine_state_code':'string',
                                'time' : 'Int64',
                                'equipment_id':'string',
                                'related_equipment':'string',
                                'location_pit':'string',
                                'material_code':'string',
                                'material_type_code':'string', 
                                'entry_type':'string', 
                                'destination':'string', 
                                'reason_code':'Int64',
                                'dispatcher_name':'string',
                                'activity_code':'Int64'
    })

    # drop un-needed cols
    dispatch_df.drop(columns=['activity','equipment_oem_model', 'cost_code', 'related_equipment_oem_model'], inplace=True)

    dispatch_df.sort_values(by=['date','shift','time'], inplace=True)

    dispatch_df['time'] = dispatch_df.apply(lambda x : (time(hour=(x['time']) + 12)) 
        if ((x['shift'] == 'DAY' and x['time'] <8) or (x['shift'] == 'NIGHT' and (x['time'] > 7 and x['time'] <12))) 
        else (time(hour=0)) if ((x['shift'] == 'NIGHT' and x['time'] ==12)) 
        else (time(hour=(x['time']))) , axis= 1)

    dispatch_df['id_shift'] = dispatch_df['date'].dt.strftime('%d%m%y') + dispatch_df['shift'].str[:1] + dispatch_df['equipment_id'] 
    dispatch_df = dispatch_df.reindex(columns=['id_shift' ,'date', 'shift', 'time', 'operator_name', 'crew', 'reason_activity',
       'engine_state_code', 'equipment_id', 'related_equipment',
       'location_pit', 'material_code', 'material_type_code', 'entry_type',
       'production_value', 'destination', 'activity_code', 'reason_code',
       'dispatcher_name'])

    dispatch_df.replace(['0:00:00','00:00:00','0'], np.nan, inplace=True)
    dispatch_df['shift'] = dispatch_df['shift'].str.capitalize()

    print("...Started dispatch splitting...")

    loads_df = dispatch_df[dispatch_df['entry_type'].str.lower() == 'load'].reset_index(drop=True).copy()
    hours_df = dispatch_df[dispatch_df['entry_type'].str.lower() == 'hours'].reset_index(drop=True).copy()
    smu_df = dispatch_df[dispatch_df['entry_type'].str.lower().str.split().str[0] == 'smu'].reset_index(drop=True).copy()


    # clean loads_df
    loads_df.drop(columns = ['reason_activity', 'engine_state_code', 'entry_type', 'material_type_code', 'activity_code'], inplace= True)
    loads_df.rename(columns={'production_value' : 'loads'}, inplace=True)
    loads_df['loads'] = loads_df['loads'].astype('Int64')
    #loads_df = loads_df[loads_df['equipment_id'].str[:3] != 'EXC' ]
    #loads_df['id_shift'] = loads_df['date'].dt.strftime('%d%m%y') + loads_df['shift'].str[:1] + loads_df['related_equipment']

    # add bcm column
    loads_df['month_year'] = loads_df['date'].dt.month_name().str[:3] + '-' + loads_df['date'].dt.year.astype('string').str[2:]
    loads_df = loads_df.merge(x[['month_year','truck_factor']],on='month_year', how='outer', copy=False)
    loads_df['bcm'] = loads_df['loads'] * loads_df['truck_factor']
    loads_df.drop(columns=['month_year', 'truck_factor'], inplace=True)
    loads_df = loads_df[loads_df['date'].isna() == False]


    # clean hours_df
    hours_df.drop(columns = ['entry_type', 'related_equipment', 'material_code', 'material_type_code', 'destination'], inplace= True)
    hours_df.rename(columns={'production_value' : 'hours'}, inplace=True)
    hours_df = hours_df[hours_df['reason_code'].isna() == False]

    # clean smu_df
    smu_df = smu_df[['date', 'shift', 'equipment_id', 'entry_type', 'production_value', 'dispatcher_name', 'id_shift']]
    smu_df = smu_df.pivot_table(index=['id_shift', 'date', 'shift', 'equipment_id', 'dispatcher_name'], columns=['entry_type'])
    smu_df.columns = smu_df.columns.droplevel().rename(None)
    smu_df = smu_df.reset_index()
    smu_df.columns=smu_df.columns.str.lower().str.replace(' ','_')
    smu_df = smu_df.reindex(columns=['id_shift', 'date', 'shift', 'equipment_id', 'dispatcher_name', 'smu_start', 'smu_end', 'smu'])
    smu_duplicates = smu_df[smu_df.duplicated(['id_shift'])]
    smu_df.drop_duplicates(subset = 'id_shift', keep='last', inplace= True, ignore_index=True)

    print("...finished dispatch splitting...")
    print("/// dispatch transformation ended ///")

    return loads_df, hours_df, smu_df, errors, smu_duplicates

In [74]:
# Transform dpr
def transform_dpr(df):
    
    print("/// dpr_df transformation started ///")
    
    master_dpr = df[2].copy()

    # Drop data > 2021
    master_dpr = master_dpr[master_dpr['date'].dt.year > 2021]

    # Select loads only
    master_dpr = master_dpr[master_dpr['entry_type'].str.lower() == 'load']

    # Create month_year col and groupby
    master_dpr['month_year'] = master_dpr['date'].dt.month_name().str[:3] + '-' + master_dpr['date'].dt.year.astype('string').str[2:]
    master_dpr.sort_values(by=['date'] , inplace=True)
    master_dpr = master_dpr.groupby(["month_year"],as_index=False, sort=False).agg(
        {
            'amount' : 'sum',
            '#loads' : 'sum',
            'truck_factor' : 'mean'
        }
    ).round(decimals=2)	

    master_dpr.columns = ['month_year', 'amount', 'loads_count', 'truck_factor']
    master_dpr.drop(columns='amount', inplace=True)
    
    print("/// dpr_df transformation ended ///")

    return master_dpr

In [75]:
# Transform Performance
def transform_performance(df):

    print("/// performance_df transformation started ///")

    performance_df = df[1].copy()

    # drop all Na
    performance_df.dropna(how='all', inplace=True)
    performance_df.replace(['0:00:00','00:00:00','0'], np.nan, inplace=True)
    performance_df = performance_df[performance_df['distance'] > 0]

    performance_df.rename(columns={'exc' : 'equipment_id'}, inplace=True)
    performance_df['shift'] = performance_df['shift'].str.capitalize()
    performance_df.sort_values(by=['date','shift','equipment_id'], inplace=True)
    performance_df['id_shift'] = performance_df['date'].dt.strftime('%d%m%y') + performance_df['shift'].str[:1] + performance_df['equipment_id'] 
    performance_df['shift'] = performance_df['shift'].str.capitalize()
    performance_df = performance_df.reindex(columns = ['id_shift', 'date', 'shift', 'equipment_id', 'distance'])
    print("/// performance_df transformation finished ///")

    return performance_df

In [76]:
# Transform Meters
def transform_meters(df):
    
    print("/// meters_df transformation started ///")
    
    meters_df = df[3].copy()
    
    meters_df.columns = meters_df.columns.str.replace('/','_')
    meters_df.drop(columns= ['id', 'comments', 'sample_tube', 'inside_wmc_boundary'], inplace=True)
    meters_df.rename(columns= {'rig' : 'equipment_id', 'hole#' : 'hole_id'}, inplace=True)
    meters_df['drilled_depth'] = pd.to_numeric(meters_df['drilled_depth'], errors='coerce')
    meters_df = meters_df[meters_df['drilled_depth'] >= 0]
    meters_df['date'] = pd.to_datetime(meters_df['date'], errors='coerce')
    meters_df = meters_df[meters_df['date'].isna() == False]
    meters_df.sort_values(by=['date','shift','equipment_id'], inplace=True)
    meters_df['id_shift'] = meters_df['date'].dt.strftime('%d%m%y') + meters_df['shift'].str[:1] + meters_df['equipment_id']
    meters_df['hammer_s_n'].replace(['BIT S/N', 'Hammer S/N'] , 'N/A', inplace=True)
    meters_df = meters_df.reindex(columns= ['id_shift', 'date', 'shift', 'crew', 'drilling_type', 'equipment_id', 'driller',
           'location', 'hole_id', 'hole_type', 'drilled_depth', 'bit_s_n',
           'hammer_s_n', 'pen_rate', 'section'])
    
    print("/// meters_df transformation ended ///")
    
    return meters_df

In [77]:
# Transform Rigs Hours
def transform_rigs_hours(df):
    
    def activity_category(x):
    
        y = 'N/A'

        if x == 'dt':
            y = 'Downtime'
        elif x == 'maint':
            y = 'Maintenance'
        elif x == 'wt':
            y = 'Worktime'
        elif x == 'sb' or x == 'standby':
            y = 'Standby'
        elif x == 'Drilling':
            y = 'Drilling'

        return y 
    
    print("/// rigs_hours_df transformation started ///")
    
    rigs_hours = df[4].copy()
    
    rigs_hours.columns = ['date', 'shift', 'rig#', 'drilling_type', 'Drilling_Drilling hrs',
       'wt_Directional drilling', 'wt_Survey', 'wt_Core orientation',
       'dt_Tramming', 'wt_Setup - Pulldown',
       'wt_Re-drill', 'wt_Condition hole',
       'wt_ream_open_out_hole', 'wt_reaming_casing',
       'wt_Freeing stuck rods - Ground conditions',
       'wt_Freeing stuck Casing - Ground conditions',
       'wt_Freeing stuck rods - Operating fault',
       'wt_RDT change',
       'wt_Change - Reduce drill mode',
       'wt_tripping_end_of_hole', 'wt_tripping_stuck_tube',
       'wt_tripping_dropped_core', 'wt_tripping_lower/retrieve_casing',
       'wt_Retrieving dropped RDT',
       'wt_tripping_broken_wireline', 'wt_cement_hole',
       'wt_RC Inner tube', 'wt_RC Sample tube',
       'wt_Checking - Changing rods', 'wt_other_work_time',
       'reason\n(text)', 'standby_Client Delay power',
       'standby_Client Water', 'standby_Client Fuel',
       'standby_Client vent - air', 'sb_Client - Blast',
       'standby_Client Access',
       'standby_Client Waiting for areas - drill pad',
       'standby_client Waiting for mark-up',
       'standby_client Waiting for client rep',
       'standby_client cement drying', 'standby_CAPITAL Mining',
       'standby_client Non chargeable', 'reason\n(text).1',
       'maint_Daily service', 'maint_Planned', 'maint_Break down',
       'reason\n(text).2', 'maint_Operator damage', 'reason\n(text).3',
       'dt_Safety meeting - Toolbox', 'dt_Pre-start - Handover',
       'dt_Housekeeping', 'dt_Waiting for CD safety',
       'dt_Waiting for CD water/fuel', 'dt_Waiting for equipment',
       'dt_Waiting for people', 'dt_Refuel', 'dt_Refilling water',
       'dt_Retrieving lost RDT', 'dt_monthly_rig_inspection',
       'dt_Travel to rig', 'dt_Weather', 'dt_Religious time',
       'dt_Lunch', 'dt_Other', 'reason\n(text).4']
    
    rigs_hours.rename(columns= {'rig#' : 'equipment_id'}, inplace=True)
    rigs_hours['date'] = pd.to_datetime(rigs_hours['date'], errors='coerce')
    rigs_hours = rigs_hours[rigs_hours['date'].isna() == False]
    rigs_hours.sort_values(by=['date','shift','equipment_id'], inplace=True)
    rigs_hours['id_shift'] = rigs_hours['date'].dt.strftime('%d%m%y') + rigs_hours['shift'].str[:1] + rigs_hours['equipment_id']  
    rigs_hours = rigs_hours[rigs_hours['drilling_type'] != '-']
    rigs_hours.dropna(axis=1, how='all', inplace=True)
    rigs_hours_melted = pd.melt(rigs_hours, id_vars= ['id_shift', 'date', 'shift', 'equipment_id', 'drilling_type'], var_name='activity_type', value_name= 'hours' )
    rigs_hours_melted = rigs_hours_melted.sort_values(by=['date', 'shift', 'equipment_id', 'activity_type'], ignore_index=True)
    rigs_hours_melted['hours'] = pd.to_numeric(rigs_hours_melted['hours'], errors='coerce')
    rigs_hours_melted = rigs_hours_melted[rigs_hours_melted['hours'] > 0]
    rigs_hours_melted['activity_category'] = rigs_hours_melted['activity_type'].str.split('_').str[0].map(activity_category)
    rigs_hours_melted['activity_type'] = rigs_hours_melted['activity_type'].str.split('_').str[1]
    print("/// rigs_hours_df transformation ended ///")
    
    return rigs_hours_melted

In [78]:
 def transform_data(df):
        
    print('\n*** START DATA TRANSFORMATION ***\n')
        
    new_dfs = general_clean(df)
    performance_df = transform_performance(new_dfs)
    dpr_df = transform_dpr(new_dfs)
    loads_df, hours_df, smu_df, errors, smu_duplicates = transform_dispatch(new_dfs, dpr_df)
    meters_df = transform_meters(new_dfs)
    rigs_hours_df = transform_rigs_hours(new_dfs)
    
    print('\n*** DATA TRANSFORMATION ENDED ***\n')

    return performance_df, dpr_df, loads_df, hours_df, smu_df, errors, meters_df, rigs_hours_df, smu_duplicates

In [86]:
dfs = list(extract_data())

*** START DATA EXTRACTION ***

Reading Dispatch Files...

dispatch files number =1

Reading dispatch\01. Database Master (february_2023 ).xlsx
Columns = 22
Master df Columns = 22
finished dispatch\01. Database Master (february_2023 ).xlsx

last dispatch date = 9682   2023-02-03
Name: DATE, dtype: datetime64[ns]

Dispatch Files Reading Ended Successfully...

Reading Performance Files...

performance files number = 1
finished performance\DPR Daily Production Report February Performance.xlsx
Columns = 4
last performance date = 249   NaT
Name: DATE, dtype: datetime64[ns]

Performance Files Reading Ended Successfully...

Reading dpr Files...

dpr files number = 1
finished dpr\DPR-Daily Production Report_February.xlsx
Columns = 5
last performance date = 20015   2023-02-03
Name: Date, dtype: datetime64[ns]

Performance Files Reading Ended Successfully...

Reading drilling Files...

daily data files number = 1
last meters date = 55973   NaT
Name: Date, dtype: datetime64[ns]
last rigs_hours dat

In [87]:
distance_df, truck_factor_df, loads_df, hours_df, smu_df, errors, meters_df, rigs_hours_df, smu_duplicates = transform_data(dfs)


*** START DATA TRANSFORMATION ***

/// performance_df transformation started ///
/// performance_df transformation finished ///
/// dpr_df transformation started ///
/// dpr_df transformation ended ///
/// dispatch transformation started ///
Converting production values to numeric...
Found and dropped 0 errors
...Started dispatch splitting...
...finished dispatch splitting...
/// dispatch transformation ended ///
/// meters_df transformation started ///
/// meters_df transformation ended ///
/// rigs_hours_df transformation started ///
/// rigs_hours_df transformation ended ///

*** DATA TRANSFORMATION ENDED ***



In [88]:
# get Equipments df
def extract_equipment_Drilling_types(df1 = dfs[0], df2 = meters_df):
    '''
    Exctract equipment models and store them in a seperate dataframe.
    by default df1 = dispatch raw df, df2 = meters_df
    '''
    def rigs_types(rig):
    
        rig_type = None
    
        if rig[3:] == '081' or rig[3:] == '067':
            rig_type = 'DM30'

        elif rig[3:] == '100' or rig[3:] == '101' or rig[3:] == '102' or rig[3:] == '115' or rig[3:] == '129':
            rig_type = 'DM45'

        elif rig[3:] == '111' or rig[3:] == '112' or rig[3:] == '122' or rig[3:] == '121' or rig[3:] == '130' or rig[3:] == '170' :
            rig_type = 'D65'

        elif rig[3:] == '116' or rig[3:] == '126' or rig[3:] == '131':
            rig_type = 'DML'

        elif rig[3:] == '140' or rig[3:] == '141' or rig[3:] == '142':
            rig_type = 'D65mkII'

        elif rig[3:] == '151' or rig[3:] == '152' or rig[3:] == '153' or rig[3:] == '154':
            rig_type = 'DR410i'

        elif rig[3:] == '125':
            rig_type = 'Terex'

        elif rig[3:] == '092' or rig[3:] == '157' or rig[3:] == '173':
            rig_type = 'Explorac'

        elif rig[3:] == '028':
            rig_type = 'Unicon'

        return rig_type


    equipment_df = pd.DataFrame(columns=['Equipment_id', 'Equipment_oem_model', 'Equipment_type', 'Equipment_department'])
    
    # first df for Mining
    equipment0 = df1[['EQUIPMENT ID', 'EQUIPMENT OEM MODEL']].drop_duplicates().reset_index(drop=True)
    equipment0 = equipment0[equipment0['EQUIPMENT ID'].isna() == False]
    equipment0['Equipment_type'] = equipment0['EQUIPMENT ID'].map(lambda x : 'Excavator'
    if x[:3] == 'EXC' else 'Truck' if x[:3] == 'HMT' else 'Grader' if x[:3] =='GRA' else 'Water_Truck' if x[:3] == 'HWT'
    else 'Dozer' if x[:2] == 'DZ' else 'Wheel_Dozer' if x[:3] == 'RDT' else 'N/A' )
    equipment0['equipment_department'] = 'Mining'
    equipment0.columns = ['Equipment_id', 'Equipment_oem_model', 'Equipment_type', 'Equipment_department']
    
    # second one for Drilling
    equipment1 = pd.DataFrame(columns=['Equipment_id', 'Equipment_oem_model', 'Equipment_type', 'Equipment_department'])
    equipment1['Equipment_id'] = meters_df['Equipment_id'].drop_duplicates()
    equipment1['Equipment_oem_model'] = equipment1['Equipment_id'].map(rigs_types)
    equipment1['Equipment_type'] = 'Rig'
    equipment1['Equipment_department'] = 'Drilling'
    
    equipment_df = pd.concat([equipment0, equipment1], ignore_index=True)
    
    # Get Drilling types df
    drilling_types_df = meters_df['Drilling_type'].drop_duplicates()
    
    return equipment_df, drilling_types_df

In [89]:
reason_codes_df = dfs[5]
final_dfs = [distance_df, truck_factor_df, loads_df, hours_df, smu_df, errors, meters_df, rigs_hours_df, reason_codes_df]
for i in range(len(final_dfs)):
    final_dfs[i].columns = final_dfs[i].columns.str.capitalize()
    final_dfs[i].columns = final_dfs[i].columns.str.replace('Date', 'Date_value')

equipment_df, drilling_types_df = extract_equipment_Drilling_types()
date_path = 'date_dimension/date_dimension.xlsx'
date_df = pd.read_excel(date_path)
shifts_df = smu_df['Shift'].drop_duplicates()


nonup_dfs = [equipment_df, drilling_types_df]
updatable_dfs = [distance_df, truck_factor_df, loads_df, hours_df, smu_df, meters_df, rigs_hours_df]

### 3 - Connect to MSSQL

In [90]:
def mssql_connect():
    conn = None
    print("ESTABLISHING A CONNECTION TO MSSQL DB...")
    try:

        connection_url = sa.engine.URL.create(
                                "mssql+pyodbc",
                                username="cl-mining-drilling-reports",
                                password="px97#@k4g#JGgApc",
                                host="capital-limited-sqlserver01.database.windows.net",
                                port=1433,
                                database="cl-mining-drilling-reports",
                                query={
                                    "driver": "ODBC Driver 18 for SQL Server",
                                    "TrustServerCertificate": "yes",
                                    "MARS_Connection": "yes"},
                            )
        sqlalch_db = sa.create_engine(connection_url, fast_executemany=True)
        insp = sa.inspect(sqlalch_db)
        conn = sqlalch_db.connect()
        print("Connected!")

        # create Loads table
        def create_tables(): 
                print("Creating Tables...")
                loads_table = sa.text(
                    "CREATE TABLE Loads ( "
                    "Id_shift varchar(20) NOT NULL,"
                    "Date_value date NOT NULL,"
                    "Shift varchar(5) NOT NULL,"
                    "Time time(0),"
                    "Operator_name varchar(max),"
                    "Crew varchar(1),"
                    "Equipment_id varchar(8) NOT NULL,"
                    "Related_equipment varchar(8),"
                    "Location_pit varchar(max),"
                    "Material_code varchar(8),"
                    "Loads int,"
                    "Destination varchar(max),"
                    "Reason_code int,"
                    "Dispatcher_name varchar(max),"
                    "Bcm float(2))"
                )
                hours_table = sa.text(
                    "CREATE TABLE Mining_hours ( "
                    "Id_shift varchar(20) NOT NULL,"
                    "Date_value date NOT NULL,"
                    "Shift varchar(5) NOT NULL,"
                    "Time time(0),"
                    "Operator_name varchar(max),"
                    "Crew varchar(1),"
                    "Reason_activity varchar(max),"
                    "Engine_state_code varchar(3),"
                    "Equipment_id varchar(8),"
                    "Location_pit varchar(max),"
                    "Hours float(6),"
                    "Activity_code int,"
                    "Reason_code int NOT NULL,"
                    "Dispatcher_name varchar(max))"
                ) 
                smu_table = sa.text(
                    "CREATE TABLE Mining_SMU ( "
                    "Id_shift varchar(20) PRIMARY KEY,"
                    "Date_value date NOT NULL,"
                    "Shift varchar(5) NOT NULL,"
                    "Equipment_id varchar(8),"
                    "Dispatcher_name varchar(max),"
                    "Smu_start float(1),"
                    "Smu_end float(1),"
                    "Smu float(1))"
                ) 
                distance_table = sa.text(
                    "CREATE TABLE Distance ( "
                    "Id_shift varchar(20) PRIMARY KEY,"
                    "Date_value date NOT NULL,"
                    "Shift varchar(5) NOT NULL,"
                    "Equipment_id varchar(8),"
                    "Distance float(2))"
                ) 
                meters_table = sa.text(
                    "CREATE TABLE Meters ( "
                    "Id_shift varchar(20) NOT NULL,"
                    "Date_value date NOT NULL,"
                    "Shift varchar(5) NOT NULL,"
                    "Crew varchar(1),"
                    "Drilling_type varchar(max),"
                    "Equipment_id varchar(8) NOT NULL,"
                    "Driller varchar(max),"
                    "Location varchar(max),"
                    "Hole_id varchar(10) NOT NULL,"
                    "Hole_type varchar(20),"
                    "Drilled_depth float(1) NOT NULL,"
                    "Bit_s_n varchar(12) NOT NULL,"
                    "Hammer_s_n varchar(8) NOT NULL,"
                    "Pen_rate float(2),"
                    "Section varchar(10) NOT NULL)"
                )  
                rigs_hours_table = sa.text(
                    "CREATE TABLE Rigs_hours ( "
                    "Id_shift varchar(20) NOT NULL,"
                    "Date_value date NOT NULL,"
                    "Shift varchar(5) NOT NULL,"
                    "Equipment_id varchar(8) NOT NULL,"
                    "Drilling_type varchar(max) NOT NULL,"
                    "Activity_type varchar(max) NOT NULL,"
                    "Hours float(2) NOT NULL,"
                    "Activity_category varchar(max) NOT NULL)"                
                ) 
                truck_factor_table = sa.text(
                    "CREATE TABLE Truck_factor ( "
                    "Month_year varchar(6) PRIMARY KEY,"
                    "Loads_count int NOT NULL,"
                    "Truck_factor float(2) NOT NULL)"
                ) 
                equipment_table = sa.text(
                    "CREATE TABLE Equipment ( "
                    "Equipment_id varchar(8) PRIMARY KEY,"
                    "Equipment_oem_model varchar(12),"
                    "Equipment_type varchar(12),"
                    "Equipment_department varchar(12))"
                )
                reason_codes_table = sa.text(
                    "CREATE TABLE Reason_codes ( "
                    "Category varchar(15) NOT NULL,"
                    "Reason_code int,"
                    "Reason_code_type varchar(12))"
                )
                date_table = sa.text(
                    "CREATE TABLE Date_dim ( "
                    "Date_value date PRIMARY KEY,"
                    "Month_name varchar(3) NOT NULL,"
                    "Day_name varchar(3) NOT NULL,"
                    "Week_of_year int NOT NULL,"
                    "Quarter int NOT NULL,"
                    "First_day_of_month date NOT NULL,"
                    "Last_day_of_month date NOT NULL)"
                )
                drilling_types_table = sa.text(
                    "CREATE TABLE Drilling_types( "
                    "Drilling_type varchar(max) PRIMARY KEY) "
                )
                shifts_table = sa.text(
                    "CREATE TABLE Shifts( "
                    "Shift varchar(5) PRIMARY KEY) "
                )


                my_tables = [loads_table, hours_table, smu_table, distance_table, 
                             meters_table, rigs_hours_table, truck_factor_table, equipment_table, 
                             drilling_types_table]

                print("\n*** CREATING TABLES ***\n")
                for table in my_tables:
                    conn.execute(table)
                    
                    '''
                    print("*** Checking if table exist ***")
                    if insp.has_table(table, schema="dbo") == False:
                        print("Table {} doesn't exist".format(table.name))
                        conn.execute(table)
                        print("Created table.")
                    
                    else : print("Table already exists.")'''

        def add_data():

            print("\n***Adding data to db***\n")

            # Define updatable tables dictionary        
            updatable_db_tables_names = ['Distance', 'Truck_factor',
                                         'Loads', 'Mining_hours', 'Mining_SMU', 'Meters','Rigs_hours']
            print("Building tables reps...\n")
            updatable_table_reps = [sa.Table(table, sa.MetaData(), autoload_with = conn) for table in updatable_db_tables_names]
            updatable_tables_dfs_dict = dict(zip(updatable_table_reps, updatable_dfs))


            for table, df in updatable_tables_dfs_dict.items():

                # delete the current month data from the db table
                # truck factor table
                print("*** Table: {} *** ".format(table.name))
                if 'Month_year' in df.columns:
                    month_year = df['Month_year'].to_list()
                    for i in month_year:
                        conn.execute(sa.delete(table=table).where(table.c.Month_year >= i)) 
                    print('Deleted {}'.format(month_year))

                    print("...Adding Data...")
                    df.to_sql(name= table.name, con=conn, schema='dbo', if_exists='append', index=False)
                    print("Added {}\n".format(table.name))

                # Other tables
                elif 'Date_value' in df.columns:
                    min_date = df['Date_value'].dt.date.min()
                    conn.execute(sa.delete(table=table).where(table.c.Date_value >= min_date))
                    print('Deleted Dates from: {} From Table: {}'.format(min_date, table.name))

                    print("...Adding Data...")
                    df.to_sql(name= table.name, con=conn, schema='dbo', if_exists='append', index=False)
                    print("Added\n")


             # Define non-updatable tables dictionary        
            nonup_db_tables_names = ['Equipment', 'Drilling_types']
            nonup_table_reps = [sa.Table(table, sa.MetaData(), autoload_with = conn) for table in nonup_db_tables_names]

            nonup_tables_dfs_dict = dict(zip(nonup_table_reps, nonup_dfs))

            for table, df in nonup_tables_dfs_dict.items():
                print("***Table: {}***".format(table.name))
                db_table = pd.DataFrame(conn.execute(sa.select(sa.text('*')).select_from(table)).fetchall())
                new_rows_df = db_table.merge(df, how='outer', indicator=True).query('_merge == "right_only"').drop(columns='_merge')


                if rows_count ==0 :
                    conn.execute(sa.delete(table=table))
                    print('Deleted table {}'.format(table.name))
                    print("...Adding Data...")
                    df.to_sql(name= table.name, con=conn, schema='dbo', if_exists='append', index=False)
                    print("Added {}\n".format(table.name))


                else:
                    print("No new Data to add")


        #create_tables() 
        add_data()               


    except Exception as error:
        print(error)
    finally:
        if conn is not None:
            print('Closing db connection...')
            conn.close()

In [91]:
mssql_connect()

ESTABLISHING A CONNECTION TO MSSQL DB...
Connected!

***Adding data to db***

Building tables reps...

*** Table: Distance *** 
Deleted Dates from: 2023-02-01 From Table: Distance
...Adding Data...
Added

*** Table: Truck_factor *** 
Deleted ['Dec-22', 'Jan-23', 'Feb-23']
...Adding Data...
Added Truck_factor

*** Table: Loads *** 
Deleted Dates from: 2023-02-01 From Table: Loads
...Adding Data...
Added

*** Table: Mining_hours *** 
Deleted Dates from: 2023-02-01 From Table: Mining_hours
...Adding Data...
Added

*** Table: Mining_SMU *** 
Deleted Dates from: 2023-02-01 From Table: Mining_SMU
...Adding Data...
Added

*** Table: Meters *** 
Deleted Dates from: 2023-02-01 From Table: Meters
...Adding Data...
Added

*** Table: Rigs_hours *** 
Deleted Dates from: 2023-02-01 From Table: Rigs_hours
...Adding Data...
Added

***Table: Equipment***
No new Data to add
***Table: Drilling_types***
No new Data to add
Closing db connection...


In [ ]:
driling_target_path = 'drilling_targets/target.xlsx'
Drilling_targets_df = pd.read_excel(driling_target_path, sheet_name='Sheet2', usecols='P:AD')
Drilling_targets_df = Drilling_targets_df.round(1)
Drilling_targets_df.replace(0,np.nan, inplace=True)
Drilling_targets_df.dropna(how='all', inplace=True)
Hole_types_df = meters_df[['drilling_type', 'hole_type']].drop_duplicates()
Hole_types_df = Hole_types_df[Hole_types_df['hole_type'] !=0]
Hole_types_df['is_Redrill'] = Hole_types_df['hole_type'].map(lambda x:True if 'Re-Drill' in x else False)
Hole_types_df.columns = Hole_types_df.columns.str.capitalize()

In [ ]:
Hole_types_table = sa.text(
    "CREATE TABLE Hole_types ( "
    "Drilling_type varchar(40) NOT NULL ,"
    "Hole_type varchar(20) PRIMARY KEY ,"
    "Is_redrill bit NOT NULL)"
)

Drilling_targets_table = sa.text(
    "CREATE TABLE Hole_types ( "
    "Month_year varchar(5) PRIMARY KEY ,"
    "Total_BH_forecast float(1) ,"
    "Total_BH_budget float(1),"
    "SGM_forecast float(1),"
    "SGM_budget float(1),"
    "WMC_forecast float(1),"
    "WMC_budget float(1),"
    "Total_BH_daily float(1),"
    "Total_BH_budget_daily float(1),"
    "SGM_forecast_daily float(1),"
    "SGM_budget_daily float(1),"
    "WMC_forecast_daily float(1),"
    "WMC_budget_daily float(1),"
    "Total_GC float(1),"
    "Total_GC_daily float(1))"
)
drilling_types_table = sa.text(
                    "CREATE TABLE Drilling_types( "
                    "Drilling_type varchar(40) PRIMARY KEY) "
                )
#conn.execute(Hole_types_table)
#conn.execute(drilling_types_table)

In [58]:
#mining_monthly_targets_path = 'mining_targets/monthly_budget_forecast.csv'
#mining_monthly_targets_df = pd.read_csv(mining_monthly_targets_path)